In [2]:
import numpy as np
import copy
from numpy import linalg as LA
import sympy
from sympy.physics.quantum.cg import CG, cg_simp
from sympy import S

In [3]:
n=5
spin=0.5


s=int(spin*2+1) # Multiplicity of the system
psi=[0]*pow(s,n) # Creating array which contains all basis states
y=[-spin]*n # Basis state with all states down

for a in range(pow(s,n)):
    x=np.base_repr(a, base=s) # Translating the number a into number system of arbitrary base s
    x = [int(x) for x in str(x)]
    for b in range(len(x)):
        y[b]=x[len(x)-b-1]-spin
    psi[a]=copy.copy(y)
    
print("Depiction of generated states:\n",psi)

Depiction of generated states:
 [[-0.5, -0.5, -0.5, -0.5, -0.5], [0.5, -0.5, -0.5, -0.5, -0.5], [-0.5, 0.5, -0.5, -0.5, -0.5], [0.5, 0.5, -0.5, -0.5, -0.5], [-0.5, -0.5, 0.5, -0.5, -0.5], [0.5, -0.5, 0.5, -0.5, -0.5], [-0.5, 0.5, 0.5, -0.5, -0.5], [0.5, 0.5, 0.5, -0.5, -0.5], [-0.5, -0.5, -0.5, 0.5, -0.5], [0.5, -0.5, -0.5, 0.5, -0.5], [-0.5, 0.5, -0.5, 0.5, -0.5], [0.5, 0.5, -0.5, 0.5, -0.5], [-0.5, -0.5, 0.5, 0.5, -0.5], [0.5, -0.5, 0.5, 0.5, -0.5], [-0.5, 0.5, 0.5, 0.5, -0.5], [0.5, 0.5, 0.5, 0.5, -0.5], [-0.5, -0.5, -0.5, -0.5, 0.5], [0.5, -0.5, -0.5, -0.5, 0.5], [-0.5, 0.5, -0.5, -0.5, 0.5], [0.5, 0.5, -0.5, -0.5, 0.5], [-0.5, -0.5, 0.5, -0.5, 0.5], [0.5, -0.5, 0.5, -0.5, 0.5], [-0.5, 0.5, 0.5, -0.5, 0.5], [0.5, 0.5, 0.5, -0.5, 0.5], [-0.5, -0.5, -0.5, 0.5, 0.5], [0.5, -0.5, -0.5, 0.5, 0.5], [-0.5, 0.5, -0.5, 0.5, 0.5], [0.5, 0.5, -0.5, 0.5, 0.5], [-0.5, -0.5, 0.5, 0.5, 0.5], [0.5, -0.5, 0.5, 0.5, 0.5], [-0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5]]


In [9]:


# Exemplary state
a=5
print(psi[a])
b=2

if(b%2==0):
    jmax=int(n*0.5+1)
else:
    jmax=int(n*0.5+0.5)

j_rep=np.array([0]*(jmax*2)) # Counting how many reps one j has
j_rep[0]=1 #for the first cgc they only have one rep
j_rep[2]=1
    
# First spin addition:
pos=0
cg0=sympy.physics.quantum.cg.CG(0.5, psi[a][pos], 0.5, psi[a][pos+1], 0, psi[a][pos+1]+psi[a][pos])
cg0=float(cg0.doit())

cg1=sympy.physics.quantum.cg.CG(0.5, psi[a][pos], 0.5, psi[a][pos+1], 1, psi[a][pos+1]+psi[a][pos])
cg1=float(cg1.doit())

print(cg0)
print(cg1)

m=np.array([0.0]*(n+4))
m[2]=psi[a][pos]+psi[a][pos+1]
b=2

#cg_emb: embedded clebsch gordan coefficients
# estimate len of array (larger than needed)
len=1
for i in range(1,n):
    if(i%2==0):
        len=len*2-1
    else:
        len=len*2

cg_emb=np.array([0.0]*len)
        
##############################################################################
# Loop for every spin in chain
for b in range(2,n): # Starting with 2 spins
    #print(j_rep)
    if(b%2==0):
        jmax=int(b*0.5)
        jmin=0
        even=1
        for i in range(jmax):
            j_rep[2*i+1]=0 #reset j reps after switching to another spin
    else:
        jmax=int(b*0.5+0.5)
        jmin=1
        even=0
        for i in range(jmax):
            j_rep[2*i]=0 #reset j reps after switching to another spin
            
    m[b+1]=m[b]+psi[a][b]
    
###############################################################################################
    # going through every j
    for j in range(jmin,jmax+1):
        # j_q is the actual quantum number, jmin/max/num are more like indices
        if(even==1):
            j_q=j #acutal quantum number j
            j_num=int(j_q*2) # index in direct sum of spin spaces
        else:
            j_q=j-0.5
            j_num=int(j_q*2)
        
        #print(j_rep[j_num])
        # Have to do double multiple iterations-> each j creates multiple states
        for c in range(j_rep[j_num]):
            if(j_q!=0):
                cg0=sympy.physics.quantum.cg.CG(j_q, m[b], 0.5, psi[a][b], j_q-0.5, m[b+1])
                cg0=float(cg0.doit())
                print(cg0)
                print("j_num und b",j_num,b)
                #cg_emb[j_num+j_rep[j_num-1]]=cg0
                j_rep[j_num-1]+=1
                print(j_rep)
                #save clebsch gordan for even and odd n
                
        
        for c in range(j_rep[j_num]):
            cg1=sympy.physics.quantum.cg.CG(j_q, m[b], 0.5, psi[a][b], j_q+0.5, m[b+1])
            cg1=float(cg1.doit())
            print(cg1)
            print("j_num und b",j_num,b)
            #cg_emb[j_num+j_rep[j_num-1]]=cg0
            j_rep[j_num-1]+=1
            print(j_rep)



#print(cg_emb)


[0.5, -0.5, 0.5, -0.5]
0.7071067811865476
0.7071067811865476
1.0
j_num und b 0 2
[1 0 1 0 0 1]
-0.5773502691896257
j_num und b 2 2
[1 1 1 0 0 1]
0.816496580927726
j_num und b 2 2
[1 2 1 0 0 1]
0.7071067811865476
j_num und b 1 3
[1 2 0 0 0 1]
0.7071067811865476
j_num und b 1 3
[2 2 0 0 0 1]
0.7071067811865475
j_num und b 1 3
[3 2 0 0 0 1]
0.7071067811865475
j_num und b 1 3
[4 2 0 0 0 1]


In [4]:
############################### try j_num_even and j_num_odd for indexing


# Exemplary state
a=1
print(psi[a])
b=2
if(b%2==0):
    jmax=int(n*0.5+1)
else:
    jmax=int(n*0.5+0.5)

j_rep=np.array([0]*(jmax*2)) # Counting how many reps one j has
j_rep[0]=1 #for the first cgc they only have one rep
j_rep[2]=1
    
# First spin addition:
pos=0
cg0=sympy.physics.quantum.cg.CG(0.5, psi[a][pos], 0.5, psi[a][pos+1], 0, psi[a][pos+1]+psi[a][pos])
cg0=float(cg0.doit())

cg1=sympy.physics.quantum.cg.CG(0.5, psi[a][pos], 0.5, psi[a][pos+1], 1, psi[a][pos+1]+psi[a][pos])
cg1=float(cg1.doit())

print(cg0)
print(cg1)

m=np.array([0.0]*(n+4))
m[2]=psi[a][pos]+psi[a][pos+1]
b=2

#cg_emb: embedded clebsch gordan coefficients
# estimate len of array (larger than needed)
len=1
for i in range(1,n):
    if(i%2==0):
        len=len*2-1
    else:
        len=len*2

cg_emb_old=np.array([0.0]*(len+5))
cg_emb_new=np.array([0.0]*(len+5))

cg_emb_old[0]=cg0
cg_emb_old[1]=cg1

##############################################################################
# Loop for every spin in chain
for b in range(2,n): # Starting with 2 spins
    print(j_rep)
    if(b%2==0):
        jmax=int(b*0.5) #WRONG J-max in j_rep reset!!!!!!!
        jmin=0
        even=1
        for i in range(jmax):
            j_rep[2*i+1]=0 #reset j reps after switching to another spin
            #cg_emb[2*i+1]=0
    else:
        jmax=int(b*0.5)
        jmin=1
        even=0
        for i in range(int(b*0.5+0.5)):
            j_rep[2*i]=0 #reset j reps after switching to another spin
            #cg_emb[2*i]=0
    print(j_rep)
    m[b+1]=m[b]+psi[a][b]
    z=0
###############################################################################################
    # going through every j
    rep=0
    for j in range(0,jmax+1):
        # j_q is the actual quantum number, jmin/max/num are more like indices

        if(even==1):
            j_q=j # actual quantum number j_q
            j_num=int(j_q*2)
        else:
            j_q=j+0.5
            j_num=int(j_q*2)
        
        #print(j_rep[j_num])
        # Have to do double multiple iterations-> each j creates multiple states
        for c in range(j_rep[j_num]):
            if(j_q!=0):
                cg0=sympy.physics.quantum.cg.CG(j_q, m[b], 0.5, psi[a][b], j_q-0.5, m[b+1])
                cg0=float(cg0.doit())
                print(cg0)
                #print("j_num und b",j_num-1,b)
                cg_emb_new[z]=cg0*cg_emb_old[rep+c] # save every single coefficient individually!!!!!!! add jreps to index of cg emb
                z+=1
                j_rep[j_num-1]+=1
                #save clebsch gordan for even and odd n
                
                #say b=2: j_rep[j_num-1] are reps for b=3, j_rep[j_num] are reps for b=2
                
        
        for c in range(j_rep[j_num]):
            cg1=sympy.physics.quantum.cg.CG(j_q, m[b], 0.5, psi[a][b], j_q+0.5, m[b+1])
            cg1=float(cg1.doit())
            print(cg1)
            #print("j_num und b",j_num+1,b)
            cg_emb_new[z]=cg1*cg_emb_old[rep+c]
            z+=1
            #print(cg_emb[j_num+1])
            j_rep[j_num+1]+=1
        
        rep+=j_rep[j_num]
            
    cg_emb_old[:]=cg_emb_new[:]
    cg_emb_new[:]=0

print(j_rep)
print(cg_emb_old)


[0.5, -0.5, -0.5, -0.5, -0.5]
0.7071067811865476
0.7071067811865476
[1 0 1 0 0 0]
[1 0 1 0 0 0]
1.0
0.5773502691896257
0.816496580927726
[1 2 1 1 0 0]
[0 2 0 1 0 0]
0.0
0.0
1.0
1.0
0.5
0.8660254037844387
[2 2 3 1 1 0]
[2 0 3 0 1 0]
0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
0.447213595499958
0.8944271909999157
[2 5 3 4 1 1]
[0.         0.         0.         0.         0.         0.70710678
 0.40824829 0.28867513 0.2236068  0.4472136  0.         0.
 0.         0.         0.         0.        ]


In [4]:
# Completely new attempt:

#Define Clebschgordan transformation matrix
T=np.array([[0.0]*pow(2,n)]*pow(2,n))

cg_plus=np.array([0.0]*pow(2,n))
cg_minus=np.array([0.0]*pow(2,n))

for b in range(n): # Starting with 2 spins
    if(b%2==0):
        jmax=int(b*0.5)
        jmin=0
        even=1
        for i in range(jmax):
            j_rep[2*i+1]=0 #reset j reps after switching to another spin
    else:
        jmax=int(b*0.5+0.5)
        jmin=1
        even=0
        for i in range(jmax):
            j_rep[2*i]=0 #reset j reps after switching to another spin
            
    m[b+1]=m[b]+psi[a][b]


NameError: name 'm' is not defined

In [6]:
for i in range(1):
    print(i)

0
